In [ ]:
import pandas as pd
import requests
import re
from datetime import datetime
import plotly.express as px
from statistics import mean

In [ ]:
link = 'https://raw.githubusercontent.com/murpi/wilddata/master/quests/beverage_dispenser.json'
r = requests.get(link)
df = pd.json_normalize(r.json(),  record_path= ["content"])
df2 = df.copy()
df2.head()

,date,product,machine
0,2019-11-14T11:53:35.000Z,refill,A
1,2019-11-14T12:06:35.000Z,coffee,A
2,2019-11-14T12:07:59.000Z,soda,A
3,2019-11-14T12:13:45.000Z,soda,A
4,2019-11-14T12:22:35.000Z,coffee,A


In [ ]:
index_refill_A = df2.loc[(df2['product'] == "refill") & (df2['machine'] == "A")]
index_refill_B = df2.loc[(df2['product'] == "refill") & (df2['machine'] == "B")]
index_refill_C = df2.loc[(df2['product'] == "refill") & (df2['machine'] == "C")]
index_refill_D = df2.loc[(df2['product'] == "refill") & (df2['machine'] == "D")]
index_refill_A

,date,product,machine
0,2019-11-14T11:53:35.000Z,refill,A
1840,2019-11-21T11:55:39.000Z,refill,A
3678,2019-11-28T11:54:41.000Z,refill,A
5515,2019-12-05T12:34:10.000Z,refill,A
7360,2019-12-12T12:21:32.000Z,refill,A
9200,2019-12-19T11:49:05.000Z,refill,A


In [ ]:
for i in range (len(index_refill_A)):
  print (index_refill_A.index[i])

0
1840
3678
5515
7360
9200


In [ ]:
df2["time"] = df2["date"].apply(lambda x: re.search('\d+:\d+:\d+', x).group(0))
df2["date"] = df2["date"].apply(lambda x: re.search('\d+-\d+-\d+', x).group(0))

def find_day(date):
  transformed_date = datetime.strptime(date, '%Y-%m-%d').strftime("%A")
  return transformed_date

df2["day"] = df2["date"].apply(find_day)
df2

,date,product,machine,time,day
0,2019-11-14,refill,A,11:53:35,Thursday
1,2019-11-14,coffee,A,12:06:35,Thursday
2,2019-11-14,soda,A,12:07:59,Thursday
3,2019-11-14,soda,A,12:13:45,Thursday
4,2019-11-14,coffee,A,12:22:35,Thursday
...,...,...,...,...,...
11037,2019-12-25,nrj,A,20:10:53,Wednesday
11038,2019-12-25,nrj,B,20:16:30,Wednesday
11039,2019-12-25,nrj,B,20:26:35,Wednesday
11040,2019-12-25,nrj,B,21:11:55,Wednesday


In [ ]:
df_test = pd.DataFrame(columns = ["product", "machine", "product_sold", "refill_number"])

index_refill = [index_refill_A, index_refill_B, index_refill_C, index_refill_D]

for v in range (len(df2["machine"].unique())):
  for i in range (len(index_refill_A)):
    if i+1 <= (len(index_refill_A)-1):
      refill_coffee = df2.loc[(df2['product'] == "coffee") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill[v].index[i]) & (df2.index < index_refill[v].index[i+1])].count()[1]
      new_row = {"product": "coffee", "machine": df2["machine"].unique()[v], "product_sold": refill_coffee, "refill_number": i+1}
      df_test = df_test.append(new_row, ignore_index=True)
      refill_soda = df2.loc[(df2['product'] == "soda") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill[v].index[i]) & (df2.index < index_refill[v].index[i+1])].count()[1]
      new_row = {"product": "soda", "machine": df2["machine"].unique()[v], "product_sold": refill_soda, "refill_number": i+1}
      df_test = df_test.append(new_row, ignore_index=True)
      refill_nrj = df2.loc[(df2['product'] == "nrj") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill[v].index[i]) & (df2.index < index_refill[v].index[i+1])].count()[1]
      new_row = {"product": "nrj", "machine": df2["machine"].unique()[v], "product_sold": refill_nrj, "refill_number": i+1}
      df_test = df_test.append(new_row, ignore_index=True)

df_test.head()


,product,machine,product_sold,refill_number
0,coffee,A,280,1
1,soda,A,120,1
2,nrj,A,57,1
3,coffee,A,280,2
4,soda,A,120,2


In [ ]:
coffee_cost = 0.3
soda_cost = 0.6
nrj_cost = 0.8

travel = 50
refill = 20

coffee_sell = 1
soda_sell = 1.6
nrj_sell = 1.8


new_list = []

for i in range(len(df_test["product"])):
  if df_test["product"][i] == "coffee":
    new_list.append(coffee_sell*df_test["product_sold"][i])
  if df_test["product"][i] == "soda":
    new_list.append(soda_sell*df_test["product_sold"][i])
  if df_test["product"][i] == "nrj":
    new_list.append(nrj_sell*df_test["product_sold"][i])

df_test["CA"] = new_list

new_list = []

for i in range(len(df_test["product"])):
  if df_test["product"][i] == "coffee":
    new_list.append(coffee_cost*df_test["product_sold"][i])
  if df_test["product"][i] == "soda":
    new_list.append(soda_cost*df_test["product_sold"][i])
  if df_test["product"][i] == "nrj":
    new_list.append(nrj_cost*df_test["product_sold"][i])

df_test["cost"] = new_list
df_CA_cost = df_test.copy()
df_CA_cost.head()

,product,machine,product_sold,refill_number,CA,cost
0,coffee,A,280,1,280.0,84.0
1,soda,A,120,1,192.0,72.0
2,nrj,A,57,1,102.6,45.6
3,coffee,A,280,2,280.0,84.0
4,soda,A,120,2,192.0,72.0


In [ ]:
df_total_per_machine_and_product = pd.DataFrame(df_test.drop(columns = ["refill_number", "CA", "cost"]))
df_total_per_machine_and_product = df_total_per_machine_and_product.groupby(['machine', "product"]).sum().reset_index()
df_total_per_machine_and_product

,machine,product,product_sold
0,A,coffee,1400
1,A,nrj,288
2,A,soda,600
3,B,coffee,1400
4,B,nrj,293
5,B,soda,600
6,C,coffee,1400
7,C,nrj,300
8,C,soda,600
9,D,coffee,1400


In [ ]:
fig = px.bar(df_total_per_machine_and_product, x='machine', y="product_sold", template='plotly_dark', color="product", text="product", 
             title = 'Total sells per machine',
             labels = {'machine': '', 'product_sold': ''}
             )
fig.update_layout(showlegend=False, title_x=0.5)
fig.show()

##<b><font color='orange'>Visualisation conclusion : all machines are usually selling all their stock before a refill happens. Only some nrj drinks are left sometimes.</font></b>

In [ ]:
df_per_day = df2[df2["product"] != "refill"]
df_per_day

,date,product,machine,time,day
1,2019-11-14,coffee,A,12:06:35,Thursday
2,2019-11-14,soda,A,12:07:59,Thursday
3,2019-11-14,soda,A,12:13:45,Thursday
4,2019-11-14,coffee,A,12:22:35,Thursday
5,2019-11-14,soda,A,12:37:44,Thursday
...,...,...,...,...,...
11037,2019-12-25,nrj,A,20:10:53,Wednesday
11038,2019-12-25,nrj,B,20:16:30,Wednesday
11039,2019-12-25,nrj,B,20:26:35,Wednesday
11040,2019-12-25,nrj,B,21:11:55,Wednesday


In [ ]:
df_per_day2 = df_per_day.groupby(["day", "product", "machine"]).size().reset_index().rename(columns={0:'valuesum'})
df_per_day2

,day,product,machine,valuesum
0,Friday,coffee,A,486
1,Friday,coffee,B,436
2,Friday,coffee,C,580
3,Friday,coffee,D,502
4,Friday,nrj,A,104
...,...,...,...,...
67,Tuesday,soda,B,21
68,Wednesday,coffee,B,3
69,Wednesday,nrj,A,25
70,Wednesday,nrj,B,42


In [ ]:
cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

df_per_day2['day'] = pd.Categorical(df_per_day2['day'], categories=cats, ordered=True)
df_per_day2 = df_per_day2.sort_values('day')
df_per_day2

,day,product,machine,valuesum
17,Monday,nrj,B,45
20,Monday,soda,B,180
16,Monday,nrj,A,24
15,Monday,coffee,D,542
14,Monday,coffee,C,529
...,...,...,...,...
40,Sunday,nrj,B,44
39,Sunday,nrj,A,26
38,Sunday,coffee,D,163
36,Sunday,coffee,B,187


In [ ]:
df_per_day3  = df_per_day2.groupby(['day', 'product']).sum().reset_index()
df_per_day3

,day,product,valuesum
0,Monday,coffee,1967.0
1,Monday,nrj,93.0
2,Monday,soda,539.0
3,Tuesday,coffee,932.0
4,Tuesday,nrj,88.0
5,Tuesday,soda,22.0
6,Wednesday,coffee,3.0
7,Wednesday,nrj,78.0
8,Wednesday,soda,NaN
9,Thursday,coffee,411.0


In [ ]:
fig = px.bar(df_per_day3, x='day', y="valuesum", template='plotly_dark', color="product", text="product", 
             title = 'Total sells per day',
             labels = {'day': '', 'valuesum': 'Sells'}
             )
fig.update_layout(showlegend=False, title_x=0.5)
fig.show()

##<b><font color='orange'>Visualisation conclusion : sells are getting lower on tuesday and wednesday probably because stocks are empty. I suspect that some nrj drinks are sold wednesday because nothing else is available.</font></b>

In [ ]:
def time_window(time):
    hour1 = int(time[:2])
    hour2 = int(time[:2])+1
    return str(hour1) + "h - " + str(hour2) + "h"

df2["time_window"] = df2["time"].apply(time_window)
df_time_window = df2[df2["product"] != "refill"]

catstime = [ '0h - 1h', '1h - 2h', '2h - 3h', '3h - 4h', '4h - 5h', '5h - 6h', '6h - 7h', '7h - 8h', '8h - 9h', '9h - 10h', '10h - 11h', '11h - 12h', '12h - 13h', '13h - 14h', '14h - 15h', 
            '15h - 16h', '16h - 17h', '17h - 18h', '18h - 19h', '19h - 20h', '20h - 21h', '21h - 22h', '22h - 23h', '23h - 24h']

df_time_window['time_window'] = pd.Categorical(df_time_window['time_window'], categories=catstime, ordered=True)
df_time_window = df_time_window.sort_values('time_window')
df_time_window

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,product,machine,time,day,time_window
10387,2019-12-23,soda,D,00:19:30,Monday,0h - 1h
737,2019-11-16,soda,B,00:06:20,Saturday,0h - 1h
7545,2019-12-13,soda,C,00:52:46,Friday,0h - 1h
7544,2019-12-13,soda,D,00:48:47,Friday,0h - 1h
7543,2019-12-13,soda,D,00:48:36,Friday,0h - 1h
...,...,...,...,...,...,...
8374,2019-12-14,soda,B,23:44:59,Saturday,23h - 24h
8375,2019-12-14,nrj,C,23:46:44,Saturday,23h - 24h
8376,2019-12-14,nrj,B,23:51:33,Saturday,23h - 24h
3675,2019-11-27,nrj,B,23:59:58,Wednesday,23h - 24h


In [ ]:
df_time_window2 = pd.DataFrame(df_time_window.drop(columns = ["date", "machine", "time"]))
df_time_window3  = df_time_window2.groupby(['time_window', 'product']).count().reset_index()
df_time_window3

,time_window,product,day
0,0h - 1h,coffee,56.0
1,0h - 1h,nrj,54.0
2,0h - 1h,soda,71.0
3,1h - 2h,coffee,75.0
4,1h - 2h,nrj,27.0
...,...,...,...
67,22h - 23h,nrj,119.0
68,22h - 23h,soda,95.0
69,23h - 24h,coffee,35.0
70,23h - 24h,nrj,86.0


In [ ]:
fig = px.bar(df_time_window3, x='time_window', y="day", template='plotly_dark', color="product", text="product", 
             title = 'Total sells per hours',
             labels = {'day': '', 'time_window': ' '}
             )
fig.update_layout(showlegend=False, title_x=0.5)
fig.update_xaxes(tickangle=45)
fig.show()

##<b><font color='orange'>Visualisation conclusion : people mostly drink coffees in the morning and nrj in the evening. The station is closed between 2am and 5am.</font></b>

In [ ]:
df_refill = df2[df2["product"] == "refill"]
df_refill = df_refill.sort_values('time')

In [ ]:
fig = px.scatter(df_refill, x="machine", y="time", template='plotly_dark', 
             title = 'Refill hours per machine',
             labels = {'time': '', 'machine': ' '}
             )
fig.update_layout(showlegend=False, title_x=0.5)
fig.show()

##<b><font color='orange'>Visualisation conclusion : refills happen around the same time each day (thursday) for each machine.</font></b>

In [ ]:
df_last_coffee = pd.DataFrame(columns = ["date", "product", "machine", "time"])
df_last_soda = pd.DataFrame(columns = ["date", "product", "machine", "time"])
df_last_nrj = pd.DataFrame(columns = ["date", "product", "machine", "time"])

for v in range (len(df2["machine"].unique())):
  for i in range (len(index_refill_A)):
    if i+1 <= (len(index_refill_A)-1):
      new_row = df2.loc[(df2['product'] == "coffee") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill_A.index[i]) & (df2.index < index_refill_A.index[i+1])].tail(1)
      df_last_coffee = df_last_coffee.append(new_row, ignore_index=True)
      new_row = df2.loc[(df2['product'] == "soda") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill_A.index[i]) & (df2.index < index_refill_A.index[i+1])].tail(1)
      df_last_soda = df_last_soda.append(new_row, ignore_index=True)
      new_row = df2.loc[(df2['product'] == "nrj") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill_A.index[i]) & (df2.index < index_refill_A.index[i+1])].tail(1)
      df_last_nrj = df_last_nrj.append(new_row, ignore_index=True)

df_last_coffee

,date,product,machine,time,day,time_window
0,2019-11-19,coffee,A,20:34:50,Tuesday,20h - 21h
1,2019-11-26,coffee,A,15:26:29,Tuesday,15h - 16h
2,2019-12-03,coffee,A,22:58:14,Tuesday,22h - 23h
3,2019-12-10,coffee,A,22:09:55,Tuesday,22h - 23h
4,2019-12-17,coffee,A,17:52:16,Tuesday,17h - 18h
5,2019-11-20,coffee,B,09:19:33,Wednesday,9h - 10h
6,2019-11-26,coffee,B,21:35:41,Tuesday,21h - 22h
7,2019-12-03,coffee,B,19:54:44,Tuesday,19h - 20h
8,2019-12-10,coffee,B,20:48:47,Tuesday,20h - 21h
9,2019-12-17,coffee,B,20:48:47,Tuesday,20h - 21h


In [ ]:
df_last_coffee['day'] = pd.Categorical(df_last_coffee['day'], categories=cats, ordered=True)
df_last_coffee = df_last_coffee.sort_values('day')

fig = px.bar(df_last_coffee, x='day', y="date", template='plotly_dark', text="machine", 
             title = 'Day of last coffee sold per machine',
             labels = {'day': ''},
              width=1200, height=600
             )
fig.update_layout(showlegend=False, title_x=0.5, yaxis={'visible': False})
fig.show()

##<b><font color='orange'>Visualisation conclusion : last coffee is usually sold on tuesday.</font></b>

In [ ]:
df_last_soda

df_last_soda['day'] = pd.Categorical(df_last_soda['day'], categories=cats, ordered=True)
df_last_soda = df_last_soda.sort_values('day')

fig = px.bar(df_last_soda, x='day', y="date", template='plotly_dark', text="machine", 
             title = 'Day of last soda sold per machine',
             labels = {'day': ' '},
             width=1200, height=600
             )
fig.update_layout(showlegend=False, title_x=0.5, yaxis={'visible': False})
fig.show()

##<b><font color='orange'>Visualisation conclusion : last soda is usually sold on monday.</font></b>

In [ ]:
df_last_nrj['day'] = pd.Categorical(df_last_nrj['day'], categories=cats, ordered=True)
df_last_nrj = df_last_nrj.sort_values('day')

fig = px.bar(df_last_nrj, x='day', y="date", template='plotly_dark', text="machine", 
             title = 'Day of last nrj sold per machine',
             labels = {'day': ' '},
             width=1200, height=600
             )
fig.update_layout(showlegend=False, title_x=0.5, yaxis={'visible': False})
fig.show()

##<b><font color='orange'>Visualisation conclusion : last nrj is usually sold on wednesday but it's interesting to see that machine C is usually emptied during saturday and sunday.</font></b>

In [ ]:
df2_without_refill = df2[df2["product"] != "refill"]

In [ ]:
df_machine_A = df_per_day2.loc[(df_per_day2["machine"] == "A")]
df_machine_B = df_per_day2.loc[(df_per_day2["machine"] == "B")]
df_machine_C = df_per_day2.loc[(df_per_day2["machine"] == "C")]
df_machine_D = df_per_day2.loc[(df_per_day2["machine"] == "D")]

In [ ]:
fig = px.bar(df_machine_A, x='day', y="valuesum", template='plotly_dark', color="product", text="product", 
             category_orders={"product": ["coffee", "nrj", "soda"], "day": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}, 
             title = 'Total Sells per day on Machine A',
             labels = {'day': ' ', "valuesum":"Sells"},
             )
fig.update_layout(showlegend=False, title_x=0.5)
fig.update_xaxes(type='category')
fig.show()

In [ ]:
fig = px.bar(df_machine_B, x='day', y="valuesum", template='plotly_dark', color="product", text="product", 
             category_orders={"product": ["coffee", "nrj", "soda"], "day": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}, 
             title = 'Total Sells per day on Machine B',
             labels = {'day': ' ', "valuesum":"Sells"},
             )
fig.update_layout(showlegend=False, title_x=0.5)
fig.update_xaxes(type='category')
fig.show()

In [ ]:
fig = px.bar(df_machine_C, x='day', y="valuesum", template='plotly_dark', color="product", text="product", 
             category_orders={"product": ["coffee", "nrj", "soda"], "day": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}, 
             title = 'Total Sells per day on Machine C',
             labels = {'day': ' ', "valuesum":"Sells"},
             )
fig.update_layout(showlegend=False, title_x=0.5)
fig.update_xaxes(type='category')
fig.show()

In [ ]:
fig = px.bar(df_machine_D, x='day', y="valuesum", template='plotly_dark', color="product", text="product", 
             category_orders={"product": ["coffee", "nrj", "soda"], "day": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}, 
             title = 'Total Sells per day on Machine D',
             labels = {'day': ' ', "valuesum":"Sells"},
             )
fig.update_layout(showlegend=False, title_x=0.5)
fig.update_xaxes(type='category')
fig.show()

##<b><font color='orange'>Visualisation conclusion : almost nothing is sold on wednesday and tuesday for each machine. Machine C seems to sell faster.</font></b>

In [ ]:
df_last_coffee = pd.DataFrame(columns = ["date", "product", "machine", "time"])
df_last_soda = pd.DataFrame(columns = ["date", "product", "machine", "time"])
df_last_nrj = pd.DataFrame(columns = ["date", "product", "machine", "time"])

for v in range (len(df2["machine"].unique())):
  for i in range (len(index_refill_A)):
    if i+1 <= (len(index_refill_A)-1):
      new_row = df2.loc[(df2['product'] == "coffee") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill_A.index[i]) & (df2.index < index_refill_A.index[i+1])].head(1)
      df_last_coffee = df_last_coffee.append(new_row, ignore_index=True)
      new_row = df2.loc[(df2['product'] == "coffee") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill_A.index[i]) & (df2.index < index_refill_A.index[i+1])].tail(1)
      df_last_coffee = df_last_coffee.append(new_row, ignore_index=True)

      new_row = df2.loc[(df2['product'] == "soda") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill_A.index[i]) & (df2.index < index_refill_A.index[i+1])].head(1)
      df_last_soda = df_last_soda.append(new_row, ignore_index=True)
      new_row = df2.loc[(df2['product'] == "soda") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill_A.index[i]) & (df2.index < index_refill_A.index[i+1])].tail(1)
      df_last_soda = df_last_soda.append(new_row, ignore_index=True)

      new_row = df2.loc[(df2['product'] == "nrj") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill_A.index[i]) & (df2.index < index_refill_A.index[i+1])].head(1)
      df_last_nrj = df_last_nrj.append(new_row, ignore_index=True)
      new_row = df2.loc[(df2['product'] == "nrj") & (df2['machine'] == df2["machine"].unique()[v]) & (df2.index >= index_refill_A.index[i]) & (df2.index < index_refill_A.index[i+1])].tail(1)
      df_last_nrj = df_last_nrj.append(new_row, ignore_index=True)

df_last_coffee.head()

,date,product,machine,time,day,time_window
0,2019-11-14,coffee,A,12:06:35,Thursday,12h - 13h
1,2019-11-19,coffee,A,20:34:50,Tuesday,20h - 21h
2,2019-11-21,coffee,A,12:05:39,Thursday,12h - 13h
3,2019-11-26,coffee,A,15:26:29,Tuesday,15h - 16h
4,2019-11-28,coffee,A,12:00:41,Thursday,12h - 13h


In [ ]:
df_last_coffee["full_time"] = df_last_coffee["date"] + " " + df_last_coffee["time"]
df_last_soda["full_time"] = df_last_soda["date"] + " " + df_last_soda["time"]
df_last_nrj["full_time"] = df_last_nrj["date"] + " " + df_last_nrj["time"]

df_last_coffee_A = pd.DataFrame(df_last_coffee[df_last_coffee["machine"] == "A"])
df_last_coffee_B = pd.DataFrame(df_last_coffee[df_last_coffee["machine"] == "B"]).reset_index()
df_last_coffee_C = pd.DataFrame(df_last_coffee[df_last_coffee["machine"] == "C"]).reset_index()
df_last_coffee_D = pd.DataFrame(df_last_coffee[df_last_coffee["machine"] == "D"]).reset_index()

df_last_soda_A = pd.DataFrame(df_last_soda[df_last_soda["machine"] == "A"])
df_last_soda_B = pd.DataFrame(df_last_soda[df_last_soda["machine"] == "B"]).reset_index()
df_last_soda_C = pd.DataFrame(df_last_soda[df_last_soda["machine"] == "C"]).reset_index()
df_last_soda_D = pd.DataFrame(df_last_soda[df_last_soda["machine"] == "D"]).reset_index()

df_last_nrj_A = pd.DataFrame(df_last_nrj[df_last_nrj["machine"] == "A"])
df_last_nrj_B = pd.DataFrame(df_last_nrj[df_last_nrj["machine"] == "B"]).reset_index()
df_last_nrj_C = pd.DataFrame(df_last_nrj[df_last_nrj["machine"] == "C"]).reset_index()
df_last_nrj_D = pd.DataFrame(df_last_nrj[df_last_nrj["machine"] == "D"]).reset_index()

In [ ]:
profit = []
missed_profit = []
win_loss = 0
new_df = pd.DataFrame()
new_df["coffee_missed_profit"] = 0

for i in range(len(df_last_coffee_A)-1):
  time2 = pd.to_datetime(df_last_coffee_A["full_time"][i+1])
  time1 = pd.to_datetime(df_last_coffee_A["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((coffee_sell-coffee_cost)*280)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0


profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["coffee_missed_profit"] = my_new_list

profit = []
missed_profit = []
win_loss = 0
for i in range(len(df_last_coffee_B)-1):
  time2 = pd.to_datetime(df_last_coffee_B["full_time"][i+1])
  time1 = pd.to_datetime(df_last_coffee_B["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((coffee_sell-coffee_cost)*280)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0

profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["coffee_missed_profit"] += my_new_list

profit = []
missed_profit = []
win_loss = 0
for i in range(len(df_last_coffee_C)-1):
  time2 = pd.to_datetime(df_last_coffee_C["full_time"][i+1])
  time1 = pd.to_datetime(df_last_coffee_C["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((coffee_sell-coffee_cost)*280)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0

profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["coffee_missed_profit"] += my_new_list

profit = []
missed_profit = []
win_loss = 0
for i in range(len(df_last_coffee_D)-1):
  time2 = pd.to_datetime(df_last_coffee_D["full_time"][i+1])
  time1 = pd.to_datetime(df_last_coffee_D["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((coffee_sell-coffee_cost)*280)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0

profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["coffee_missed_profit"] += my_new_list

In [ ]:
profit = []
missed_profit = []
win_loss = 0
new_df["soda_missed_profit"] = 0

for i in range(len(df_last_soda_A)-1):
  time2 = pd.to_datetime(df_last_soda_A["full_time"][i+1])
  time1 = pd.to_datetime(df_last_soda_A["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((soda_sell-soda_cost)*280)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0


profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["soda_missed_profit"] = my_new_list

profit = []
missed_profit = []
win_loss = 0
for i in range(len(df_last_soda_B)-1):
  time2 = pd.to_datetime(df_last_soda_B["full_time"][i+1])
  time1 = pd.to_datetime(df_last_soda_B["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((soda_sell-soda_cost)*120)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0

profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["soda_missed_profit"] += my_new_list

profit = []
missed_profit = []
win_loss = 0
for i in range(len(df_last_soda_C)-1):
  time2 = pd.to_datetime(df_last_soda_C["full_time"][i+1])
  time1 = pd.to_datetime(df_last_soda_C["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((soda_sell-soda_cost)*120)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0

profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["soda_missed_profit"] += my_new_list

profit = []
missed_profit = []
win_loss = 0
for i in range(len(df_last_soda_D)-1):
  time2 = pd.to_datetime(df_last_soda_D["full_time"][i+1])
  time1 = pd.to_datetime(df_last_soda_D["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((soda_sell-soda_cost)*120)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0

profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["soda_missed_profit"] += my_new_list

In [ ]:
profit = []
missed_profit = []
win_loss = 0
new_df["nrj_missed_profit"] = 0

for i in range(len(df_last_nrj_A)-1):
  time2 = pd.to_datetime(df_last_nrj_A["full_time"][i+1])
  time1 = pd.to_datetime(df_last_nrj_A["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((nrj_sell-nrj_cost)*60)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0


profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["nrj_missed_profit"] = my_new_list

profit = []
missed_profit = []
win_loss = 0
for i in range(len(df_last_nrj_B)-1):
  time2 = pd.to_datetime(df_last_nrj_B["full_time"][i+1])
  time1 = pd.to_datetime(df_last_nrj_B["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((nrj_sell-nrj_cost)*60)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0

profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["nrj_missed_profit"] += my_new_list

profit = []
missed_profit = []
win_loss = 0
for i in range(len(df_last_nrj_C)-1):
  time2 = pd.to_datetime(df_last_nrj_C["full_time"][i+1])
  time1 = pd.to_datetime(df_last_nrj_C["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((nrj_sell-nrj_cost)*60)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0

profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["nrj_missed_profit"] += my_new_list

profit = []
missed_profit = []
win_loss = 0
for i in range(len(df_last_nrj_D)-1):
  time2 = pd.to_datetime(df_last_nrj_D["full_time"][i+1])
  time1 = pd.to_datetime(df_last_nrj_D["full_time"][i])
  time_delta = time2 - time1
  if win_loss == 0:
    profit.append(((nrj_sell-nrj_cost)*60)/((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9]))))
    win_loss +=1
  else:
    missed_profit.append((int(str(time_delta)[0])*24) + (int(str(time_delta)[7:9])))
    win_loss = 0

profit_per_hour = mean(profit)
my_new_list = [i * profit_per_hour for i in missed_profit]
new_df["nrj_missed_profit"] += my_new_list

In [ ]:
new_df2 = new_df/4
new_df2["total_missing_per_refill"] = new_df2["coffee_missed_profit"] + new_df2["soda_missed_profit"] + new_df2["nrj_missed_profit"]
new_df2["refill_cost"] = 50 + 4 * 20
new_df2["money_missed"] = new_df2["total_missing_per_refill"] - new_df2["refill_cost"]
new_df2

,coffee_missed_profit,soda_missed_profit,nrj_missed_profit,total_missing_per_refill,refill_cost,money_missed
0,65.829406,84.713235,31.298193,181.840833,130,51.840833
1,64.192119,87.797400,26.142113,178.131632,130,48.131632
2,65.861253,54.533220,30.557698,150.952170,130,20.952170
3,72.180167,57.233573,32.988766,162.402506,130,32.402506


##<b><font color='orange'>Conclusion : my final calcul is simplistic. I take the average turnover per hour between a refill and last sell of each product to find how much money is missed between last sell of each product and the following refill. While it's not perfect, I think this calcul is good enough overall. According to my calcul, we would be able to generate more income by doing a second refill. The best day would be sunday because machine C is already emptied of nrj drinks and soda starting monday.</font></b>